In [ ]:
# as png
# %matplotlib inline 
#  interactable inside ide
# %matplotlib widget
### interactable seperate window
%matplotlib tk 

import mne
import numpy as np
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')
from scipy.io import loadmat

In [ ]:
# def extractBCI3(person_id = [0]):
person_id = [0]
files = ['/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_aa_mat/100Hz/data_set_IVa_aa.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_al_mat/100Hz/data_set_IVa_al.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_av_mat/100Hz/data_set_IVa_av.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_aw_mat/100Hz/data_set_IVa_aw.mat',
'/media/mangaldeep/HDD3/DataSets/BCI3_Competition/data_set_IVa_ay_mat/100Hz/data_set_IVa_ay.mat',]

dat = loadmat(files[1], struct_as_record=True)
extra_ch =['AFp1', 'AFp2', 'FAF5', 'FAF1', 'FAF2', 'FAF6', 'FFC7', 'FFC5', 'FFC3', 'FFC1', 'FFC2', 'FFC4', 'FFC6', 
    'FFC8', 'CFC7', 'CFC5', 'CFC3', 'CFC1', 'CFC2', 'CFC4', 'CFC6', 'CFC8', 'CCP7', 'CCP5', 'CCP3', 'CCP1', 'CCP2',
    'CCP4', 'CCP6', 'CCP8', 'PCP7', 'PCP5', 'PCP3', 'PCP1', 'PCP2', 'PCP4', 'PCP6', 'PCP8', 'PPO7', 'PPO5', 'PPO1',
    'PPO2', 'PPO6', 'PPO8', 'OPO1', 'OPO2', 'OI1', 'OI2', 'I1', 'I2']

In [ ]:
sfreq = dat['nfo']['fs'][0][0][0][0]
EEGdata   = dat['cnt'].T
EEGdata.astype('float64')
EEGdata = EEGdata*0.1
nchannels, nsamples = EEGdata.shape

chan_names = [s[0] for s in dat['nfo']['clab'][0][0][0]]

event_onsets  = dat['mrk'][0][0][0][0]
event_codes   = dat['mrk'][0][0][1][0]
# event_codes = np.nan_to_num(event_codes)

labels = np.zeros((1, nsamples), int)
labels[0, event_onsets] = event_codes

cl_lab = [s[0] for s in dat['mrk']['className'][0][0][0]]
cl_lab.append('nan')
cl1    = cl_lab[0]
cl2    = cl_lab[1]

# digitized electrode positions 
xpos = dat['nfo']['xpos']
ypos = dat['nfo']['ypos']

nclasses = len(cl_lab)
nevents = len(event_onsets)

# Print some information
print('Shape of EEG:', EEGdata.shape)
print('Sample rate:', sfreq)
print('Number of channels:', nchannels)
print('Channel names:', chan_names)
print('Number of events:', len(event_onsets))
print('Event codes:', np.unique(event_codes))
print('Class labels:', cl_lab)
print('Number of classes:', nclasses)

In [ ]:
# Dictionary to store the trials in, each class gets an entry
trials = {}

# The time window to extract for each trial, here 0. -- 3.5 seconds
win = np.arange(int(0*sfreq), int(3.5*sfreq))

# Length of the time window
nsamples = len(win)

# Loop over the classes (right, foot)
for cl, code in zip(cl_lab, np.unique(event_codes)):
    
    # Extract the onsets for the class
    cl_onsets = event_onsets[event_codes.astype('str') == str(code)]
    
    # Allocate memory for the trials
    trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets)))
    
    # Extract each trial
    for i, onset in enumerate(cl_onsets):
        trials[cl][:,:,i] = EEGdata[:, win+onset]
        
# the dimensionality of the data (channels x time x trials)
print('Shape of trials[cl1]:', trials[cl1].shape)
print('Shape of trials[cl2]:', trials[cl2].shape)
# print('Shape of trials[00]:', trials[0].shape)

In [ ]:
right_hand  = np.rollaxis(trials[cl1], 2, 0)  
foot = np.rollaxis(trials[cl2], 2, 0) 
test = np.rollaxis(trials['nan'], 2, 0) 
data = np.concatenate([right_hand, foot, test])

In [ ]:
Y = np.concatenate([-np.ones(right_hand.shape[0]),
                     np.ones(foot.shape[0]),
                     np.zeros(test.shape[0])])
data.shape

In [ ]:
# Initialize an info structure
info = mne.create_info(
        ch_names = chan_names,
        ch_types = ['eeg']*nchannels,
        sfreq    = sfreq )  
# info.set_montage('standard_1020')
print('Event created :', info)


# Electrode Locations
xpos = dat['nfo']['xpos'][0][0]
ypos = dat['nfo']['ypos'][0][0]
layout_pos = np.concatenate([xpos, ypos], axis = 1)
layout = mne.channels.generate_2d_layout(
    xy = layout_pos,
    ch_names=chan_names,
    name ='EEG custom layout',
    )

In [ ]:
ten_twenty_montage = mne.channels.make_standard_montage('standard_1020')
# print(ten_twenty_montage.ch_names)
# print(chan_names)

In [ ]:
raw = mne.io.RawArray(EEGdata, info)
raw = raw.pick_channels(ch_names = ten_twenty_montage.ch_names).copy()
raw.set_montage('standard_1020')
scale = dict(mag=1e-12, grad=4e-11, eeg=100e-6)
rawfltrd = raw.filter(1, 30, verbose= False, fir_design='firwin', skip_by_annotation='edge').copy()

In [ ]:
# raw.plot_psd();

### Artifact Removal

In [ ]:
rawfltrd_car = rawfltrd.copy()
rawfltrd_car.set_eeg_reference(ref_channels='average', projection=True);

In [ ]:
rawfltrd_ssp = rawfltrd_car.copy()
eog_proj, events = mne.preprocessing.compute_proj_eog(rawfltrd_ssp, n_grad=0, n_mag=0, n_eeg=2, average=True, verbose=False, ch_name = 'Fpz', reject=None) # returns EOG Proj and events of blinks
rawfltrd_ssp.add_proj(projs=eog_proj);

In [ ]:
# rawfltrd_ssp.plot(scalings = scale);

In [ ]:
# cov = mne.Covariance()
rawfltrd_ica = rawfltrd_ssp.apply_proj().copy()
ica = mne.preprocessing.ICA(n_components = 20, noise_cov= None, random_state=2, method='picard',max_iter=500)
# Create an instance of RAW
# rawfltrd_ica = raw.copy()
# rawfltrd_ica.apply_proj()
ica.fit(rawfltrd_ica);
# n_components  = 10 then ica.exclude = [1,2]
ica.exclude = []
# Using EOG Channel to select ICA Components
ica.exclude , ex_scores = ica.find_bads_eog(rawfltrd_ica, ch_name=['Fpz','Fp1','Fp2']);#,threshold=2);

In [ ]:
ica.exclude = [0,3,5]

In [ ]:
ica.plot_sources(rawfltrd_ica);
ica.plot_scores(ex_scores);

In [ ]:
rawfltrd_ica = ica.apply(rawfltrd_ica)

In [ ]:
rawfltrd.plot_psd();

### Epoching

In [ ]:
event_id = dict(right = -1, foot = 1, test = 0)
eventLength = Y.shape[0]
ev = dat['mrk']['pos'][0][0][0] #[i*sfreq*3 for i in range(eventLength)]

events = np.column_stack((np.array(ev,  dtype = int),
                          np.zeros(eventLength,  dtype = int),
                          np.array(Y,  dtype = int)))

In [ ]:
rawepochs = rawfltrd_ica.apply_proj().copy()
epoch = mne.Epochs(rawepochs, events, event_id, tmax = 1.0)#, tmin = -3.0)
# epoch = mne.EpochsArray(data, info, events,tmin=0.0, event_id= event_id)
epoch.apply_baseline()

In [ ]:
T1 = epoch['right']
T2 = epoch['foot']
t1 = epoch['right'].average()
t2 = epoch['foot'].average()
picks=['C1', 'C2','C3', 'C4', 'C5', 'C6', 'Cz']#,'T9','T10']

In [ ]:
clas = 'right'
# epoch['right'].plot_image(combine = 'mean', title = 'right');
epoch[clas].average().plot_image(show_names = 'all', titles = clas, picks = picks);
epoch[clas].plot_topo_image(layout=layout, title = clas);
epoch[clas].plot_image(title = clas, combine = 'mean');
epoch[clas].plot_psd();


In [ ]:
clas = 'right'
epoch[clas].average().plot_image(show_names = 'all',titles = clas, picks = picks);

### Feature Extraction

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

#### Power

In [ ]:
from mne.time_frequency import tfr_morlet

In [ ]:
frequencies =  np.arange(5,30,1) #np.logspace(*np.log10([5, 30]), num=25)
chpick = [1] #baseline = (-0.5, 0.),

In [ ]:
show_names = 'all'
    # power.plot(picks=[10],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
    power.plot_joint(picks = ['C1','C3','C5'], timefreqs = [(0.19,10),(2.47,10)],mode = 'logratio', title = f'{list(T1.event_id.keys())[0]}')

In [ ]:
power = tfr_morlet(epoch[clas], freqs = frequencies, n_cycles = frequencies/2, return_itc= False)
for pick in chpick:
    # power.plot(picks=[pick],  vmin=-0.00003, vmax=0.00003, title=f'{pick}')#, mode='logratio')
    power.plot_joint(mode = 'mean', title = clas)

In [ ]:
power = tfr_morlet(T2, freqs = frequencies, n_cycles = frequencies/2, return_itc= False, average = False)
# avg = power.average()
# power.plot_topomap();
# power.plot(picks = ['C2','C4','C6'], mode = 'mean');

In [ ]:
power.data.shape

In [ ]:
t1.plot_image(picks = ['C1','C3','C5','C2','C4','C6'],show_names = 'all');

In [ ]:
T1.plot_image(combine = 'mean');

In [ ]:
power.plot_topomap();

#### WST

In [ ]:
from kymatio.sklearn import Scattering2D
from kymatio.sklearn import Scattering1D
from kymatio.scattering2d.filter_bank import filter_bank
# from numpy.fft import fft
import matplotlib.pyplot as plt
# import numpy as np
# %matplotlib tk 

In [ ]:
M,N = epoch.get_data().shape[1:]
J,L = 3,8
scat2D = Scattering2D(J=J,shape=(M,N),L=L)
data = epoch['right','foot'].get_data()
data = data.reshape(224,-1)

In [ ]:
# Do this for every epoch 
Sx = scat2D(data)

In [ ]:
train_data = epoch['right', 'foot'].get_data()
train_data = train_data.reshape(train_data.shape[0],-1)
labels = np.concatenate([-np.ones(right_hand.shape[0]),
                     np.ones(foot.shape[0])])

In [ ]:
print(train_data.shape)
print(labels.shape)

In [116]:
clf = LDA()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

array([0.71111111, 0.66666667, 0.66666667, 0.71111111, 0.88888889,
       0.77777778, 0.66666667, 0.73333333, 0.68888889, 0.62222222])

In [117]:
clf = SVC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

array([0.62222222, 0.71111111, 0.66666667, 0.66666667, 0.82222222,
       0.62222222, 0.77777778, 0.66666667, 0.71111111, 0.73333333])

In [118]:
clf = DTC()
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(train_data, labels)
pipe = Pipeline([('scatter', scat2D), ('clf', clf)])
scores = cross_val_score(pipe, train_data, labels, cv = cv, verbose=False)
scores

array([0.55555556, 0.71111111, 0.66666667, 0.71111111, 0.64444444,
       0.66666667, 0.6       , 0.57777778, 0.75555556, 0.64444444])

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
pipe.fit(epoch['right', 'foot'].get_data(), labels)
pipe.predict(epoch['test'][0].get_data())

In [ ]:
ans = []
for i in range(epoch['test'].get_data().shape[0]):
    ans.append(pipe.predict(epoch['test'][i].get_data()))

In [ ]:
ans

In [ ]:
epoch['right', 'foot'].get_data().shape

In [ ]:
labels.shape